In [ ]:
import pandas as pd
import plistlib
from pandas import DataFrame
import numpy as np
import math

In [ ]:
gbv_call_data_2020=pd.read_csv(r'/Users/angelaivonnesalgadobeltran/Documents/IronHack/Final project/Argentina_data/linea144-2020.csv')

In [ ]:
gbv_call_data_2021=pd.read_csv(r'/Users/angelaivonnesalgadobeltran/Documents/IronHack/Final project/Argentina_data/linea144-2021_0.csv')

In [ ]:
gbv_call_data_2022_1=pd.read_csv(r'/Users/angelaivonnesalgadobeltran/Documents/IronHack/Final project/Argentina_data/linea144-1ersemestre-2022.csv')

1. Apped all the dataframes from 2020, 2021 and 2022 in one Dataframe

In [ ]:
gvb_calls_data=gbv_call_data_2020.append([gbv_call_data_2021, gbv_call_data_2022_1])
gvb_calls_data

In [ ]:
gvb_calls_data

# --------Cleaning---------

In [ ]:
gvb_calls_data.isna().sum()

1. Drop "pais_nacimiento_persona_en_situacion_de_violencia" column because more than 30% of mv

In [ ]:
gvb_calls_data.drop(columns=['pais_nacimiento_persona_en_situacion_de_violencia'], inplace=True)

2. Droping rowns with mv in column genero_de_la_persona_agresora

In [ ]:
gvb_calls_data = gvb_calls_data[gvb_calls_data['genero_de_la_persona_agresora'].notna()]

3. Drop rows with mv in column 'vinculo_con_la_persona_agresora

In [ ]:
gvb_calls_data = gvb_calls_data[gvb_calls_data['vinculo_con_la_persona_agresora'].notna()]

In [ ]:
gvb_calls_data.isna().sum()

4. Cleaning mv in the column "edad_persona_en_situacion_de_violencia"

4.1 Firstly we have to clean the column 'prov_residencia_persona_en_situacion_violencia' in our general dataframe to avoid str problems with the names of the states

In [ ]:
gvb_calls_data['prov_residencia_persona_en_situacion_violencia'] = np.where((gvb_calls_data['prov_residencia_persona_en_situacion_violencia'] == 'Ciudad Autónoma de Buenos Aires'), 'Buenos Aires', gvb_calls_data['prov_residencia_persona_en_situacion_violencia'])
gvb_calls_data['prov_residencia_persona_en_situacion_violencia'] = np.where((gvb_calls_data['prov_residencia_persona_en_situacion_violencia'] == 'Santiago Del Estero'), 'Santiago del Estero', gvb_calls_data['prov_residencia_persona_en_situacion_violencia'])
gvb_calls_data['prov_residencia_persona_en_situacion_violencia'] = gvb_calls_data['prov_residencia_persona_en_situacion_violencia'].replace(np.nan, 'Buenos Aires')

4.2  We want to know wich are the rows that have mv in the column "edad_persona_en_situation_de_violencia" in order to classifie them depending of the place of occurrence. The objective is yo replace the mv by the mean of the age in each state.

In [ ]:
mv_edad :DataFrame =gvb_calls_data.loc[gvb_calls_data['edad_persona_en_situacion_de_violencia'].isna()]
mv_edad
mv_edad['prov_residencia_persona_en_situacion_violencia'].value_counts()

4.3 We create a new dataframe that contains the mean of the age in each state. After that, we merge this small dataframe with our general dataframe in order to have one column with the mean of age depending of the state. 

In [ ]:
mean_age : DataFrame=gvb_calls_data.groupby(['prov_residencia_persona_en_situacion_violencia']).agg({"edad_persona_en_situacion_de_violencia" :"mean"})
mean_age=mean_age.reset_index()
mean_age.rename(columns={'edad_persona_en_situacion_de_violencia':'media_edad'}, inplace=True)
gvb_calls_data=pd.merge(gvb_calls_data, mean_age, on='prov_residencia_persona_en_situacion_violencia')
gvb_calls_data

In [ ]:
gvb_calls_data

4.4 We replace the mv in the age columns with the age_mean of each state 

In [ ]:
def compute_age(age,mean):
    if math.isnan(age):
        return mean
    return age  

gvb_calls_data['edad']=gvb_calls_data.apply(lambda row: compute_age(row['edad_persona_en_situacion_de_violencia'], row['media_edad']), axis=1)

5. Drop column 'edad_persona_en_situacion_de_violencia'

In [ ]:
gvb_calls_data.drop(columns=['edad_persona_en_situacion_de_violencia'], inplace=True)

6. Drop rows with mv in 'genero_persona_en_situacion_de_violencia'

In [ ]:
gvb_calls_data = gvb_calls_data[gvb_calls_data['genero_persona_en_situacion_de_violencia'].notna()]

In [ ]:
gvb_calls_data.isna().sum()